<a href="https://colab.research.google.com/github/Knell999/ML_Ops_HK-Toss/blob/main/Chapter_1_%EA%B8%B0%EC%B4%88/02_%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EB%8F%84%EA%B5%AC/03_%ED%81%AC%EB%A1%A4%EB%A7%81/%5B%EC%8B%A4%EC%8A%B5%5D_REST_API_%ED%99%9C%EC%9A%A9_%EC%B9%B4%EC%B9%B4%EC%98%A4%EB%A7%B5_%EB%A6%AC%EB%B7%B0_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
REST_API_KEY = "3032411cf691dff08e2c264bf4005aeb"

In [ ]:
KEYWORD_LOCAL_URL = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}&page={}"

stations = ["광화문역"]

In [ ]:
headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

response = requests.get(KEYWORD_LOCAL_URL.format(stations[0] + " 맛집", 1), headers=headers)
dic = response.json()['documents']
dic

[{'address_name': '서울 종로구 청진동 183-1',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 한식 > 해장국',
  'distance': '',
  'id': '27540102',
  'phone': '02-735-1690',
  'place_name': '청진옥',
  'place_url': 'http://place.map.kakao.com/27540102',
  'road_address_name': '서울 종로구 종로3길 32',
  'x': '126.97944890786793',
  'y': '37.571717650682224'},
 {'address_name': '서울 중구 정동 1-48',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 한식 > 국밥',
  'distance': '',
  'id': '2067124772',
  'phone': '02-738-5688',
  'place_name': '광화문국밥 본점',
  'place_url': 'http://place.map.kakao.com/2067124772',
  'road_address_name': '서울 중구 세종대로21길 53',
  'x': '126.975391668232',
  'y': '37.5689589279236'},
 {'address_name': '서울 종로구 종로1가 24',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 한식',
  'distance': '',
  'id': '21326655',
  'phone': '02-732-1954',
  'place_name': '광화문미진 본점',
  'place_url':

In [ ]:
stores_id = [item["id"] for item in dic]
stores_id

['27540102',
 '2067124772',
 '21326655',
 '15431872',
 '967729291',
 '444516464',
 '24725284',
 '27187492',
 '152172304',
 '18363906',
 '1346301870',
 '13314584',
 '7937472',
 '18306071',
 '11064200']

In [ ]:
place_names = [item["place_name"] for item in dic]
len(place_names)

15

In [ ]:
url = f"https://place.map.kakao.com/commentlist/v/20631070"
response = requests.get(url)
data = response.json()
data

{'comment': {'kamapComntcnt': 25,
  'scoresum': 83,
  'scorecnt': 25,
  'list': [{'commentid': '11620386',
    'contents': '진짜 별로 기분만 안좋아짐 친절 위생 다 별로',
    'point': 1,
    'username': '뭉',
    'profile': '',
    'profileStatus': 'S',
    'photoCnt': 0,
    'likeCnt': 0,
    'kakaoMapUserId': '2924658141',
    'photoList': [],
    'ownerReply': {},
    'strengths': [{'id': 2, 'name': '친절'}],
    'userCommentCount': 2,
    'userCommentAverageScore': 1.0,
    'myStorePick': False,
    'level': {'nowLevel': 3, 'badge': '01'},
    'date': '2024.12.14.',
    'isMy': False,
    'isBlock': False,
    'isEditable': False,
    'isMyLike': False},
   {'commentid': '11542128',
    'contents': '하도 유명해서,포장해서 먹었는데~\n그닥!?맛집까지는 에바..',
    'point': 2,
    'username': '최악인곳만쓴다',
    'profile': '',
    'profileStatus': 'S',
    'photoCnt': 0,
    'likeCnt': 0,
    'kakaoMapUserId': '3697882697',
    'photoList': [],
    'ownerReply': {},
    'userCommentCount': 84,
    'userCommentAverageScore': 1.5,
    

In [ ]:
COMMENT_URL = "https://place.map.kakao.com/m/commentlist/v/{}/{}?order=USEFUL&onlyPhotoComment=false"

In [ ]:
all_comments = []

for id in stores_id[:2]:

  # 첫 번째 코멘트의 id는 무조건 0
  comment_id = 0

  # has_next가 true면 계속 수집한다. true인 경우 마지막 코멘트 id를 새롭게 넣어서 스크래이핑
  has_next = True

  while has_next:
    # 수집해야 할 URL 생성
    SCRAP_COMMENT_URL = COMMENT_URL.format(id, comment_id)

    # 만들어진 url로 요청
    response = requests.get(SCRAP_COMMENT_URL)

    # 전체 코멘트 정보
    comment_datas = response.json()['comment']

    # 코멘트 리스트
    comment_list  = comment_datas['list']

    all_comments.extend(comment_list)

    # 다음 페이지 존재 여부 확인
    has_next = comment_datas['hasNext']

    # has_next가 True인 경우 마지막 코멘트의 id를 comment_id로 설정
    if has_next:
      comment_id = comment_list[-1]['commentid']

    print(comment_list)

[{'commentid': '11889933', 'contents': '애기때 청진동에 있던 때부터 부모님이 데려 갔던 \n평생 서울을 대표 하는 맛집 중 하나입니다 \n기본적으로 선지해장국 하면, 빨간 국물 베이스가 많은데 다른 지역 어디에서도 이런 맛의 선지국은 찾기 힘들죠\n안에 들어간 선지.양.야채 양 보면 가성비도 좋습니다.\n김구.윤보선.김영삼 맛집으로도 유명하죠.\n원로 코매디언들이나 탤런트들도 자주 볼 수 있습니다.', 'point': 5, 'username': 'JH', 'profile': 'http://th-p.talk.kakao.co.kr/th/talkp/wkYmCc2zmW/mUYRb8jS1JdAR3Jun4lE8K/hihl02.jpg', 'profileStatus': 'S', 'photoCnt': 0, 'likeCnt': 0, 'kakaoMapUserId': '957356001', 'photoList': [], 'ownerReply': {}, 'userCommentCount': 27, 'userCommentAverageScore': 3.1, 'myStorePick': False, 'level': {'nowLevel': 18, 'badge': '02'}, 'date': '2025.01.11.', 'isMy': False, 'isBlock': False, 'isEditable': False, 'isMyLike': False}, {'commentid': '11888676', 'point': 5, 'username': 'ROSÉ Kim[김미옥]', 'profile': 'https://p.kakaocdn.net/th/talkp/wpmER4dKvI/eNUoz2AeF9Dnf3dtlmj83k/lozuca_640x640_s.jpg', 'profileStatus': 'S', 'photoCnt': 0, 'likeCnt': 0, 'kakaoMapUserId': '385829074', 'photoList': [], 'ownerReply': {}, 'strength

In [ ]:
import pandas as pd

comment_df = pd.DataFrame(all_comments)
comment_df.head()

,commentid,contents,point,username,profile,profileStatus,photoCnt,likeCnt,kakaoMapUserId,photoList,...,userCommentAverageScore,myStorePick,level,date,isMy,isBlock,isEditable,isMyLike,strengths,thumbnail
0,11889933,애기때 청진동에 있던 때부터 부모님이 데려 갔던 \n평생 서울을 대표 하는 맛집 중...,5,JH,http://th-p.talk.kakao.co.kr/th/talkp/wkYmCc2z...,S,0,0,957356001,[],...,3.1,False,"{'nowLevel': 18, 'badge': '02'}",2025.01.11.,False,False,False,False,NaN,NaN
1,11888676,NaN,5,ROSÉ Kim[김미옥],https://p.kakaocdn.net/th/talkp/wpmER4dKvI/eNU...,S,0,0,385829074,[],...,4.9,False,"{'nowLevel': 33, 'badge': '03'}",2025.01.11.,False,False,False,False,"[{'id': 2, 'name': '친절'}, {'id': 3, 'name': '분...",NaN
2,11868980,해장국의 최고존엄,5,킴주앙,http://k.kakaocdn.net/dn/cD0m9b/btqyS9qQAqR/ad...,S,1,0,1207352404,[{'url': 'http://t1.daumcdn.net/local/kakaomap...,...,4.9,False,"{'nowLevel': 31, 'badge': '03'}",2025.01.08.,False,False,False,False,"[{'id': 5, 'name': '맛'}]",http://t1.daumcdn.net/local/kakaomapPhoto/revi...
3,9354711,"평범한것 같은데, 호불호가 있는듯..",2,멍뭉이,http://t1.daumcdn.net/local/kakaomapPhoto/prof...,S,0,4,994921397,[],...,3.8,False,"{'nowLevel': 6, 'badge': '01'}",2025.01.08.,False,False,False,False,NaN,NaN
4,11830776,선지해장국이랑 모듬수육 주문했습니다\n해장국 역시 소문대로 일품입니다.,4,김지회,http://k.kakaocdn.net/dn/jQ9Vj/btqA6hs5Y5l/ZfY...,S,0,0,1271200074,[],...,4.2,False,"{'nowLevel': 4, 'badge': '01'}",2025.01.04.,False,False,False,False,"[{'id': 5, 'name': '맛'}, {'id': 3, 'name': '분위...",NaN
